In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


## Initializing the variables

In [3]:
img =['img1']
command_params = ['cp','mv','rm']
cpu_memory_vars = dict()

## Populating the variables

In [4]:
list2  = ['x86-32','x86-64']
list3 = ['1 GB','1 GB']
list4 =  ['200 MB','250 MB']


cpu_memory_vars = {'CPU':[],'Memory':[],'Hard Disk':[]}
cpu_memory_vars['CPU'].append(list2)
cpu_memory_vars['Memory'].append(list3)
cpu_memory_vars['Hard Disk'].append(list4)
cpu_memory_vars.values()


dict_values([[['x86-32', 'x86-64']], [['1 GB', '1 GB']], [['200 MB', '250 MB']]])

In [5]:
Var = []
env_vars = dict()

for a in os.environ:
    val = os.getenv(a)
    list1 = [a,val]
    for line in list1:    
        Var = list1[0]   
        Value = list1[1]  
        for line in list1:  
            env_vars.setdefault(Var, []).append(Value)
    

In [6]:
env_vars

{'ALLUSERSPROFILE': ['C:\\ProgramData',
  'C:\\ProgramData',
  'C:\\ProgramData',
  'C:\\ProgramData'],
 'APPDATA': ['C:\\Users\\suneelnair1\\AppData\\Roaming',
  'C:\\Users\\suneelnair1\\AppData\\Roaming',
  'C:\\Users\\suneelnair1\\AppData\\Roaming',
  'C:\\Users\\suneelnair1\\AppData\\Roaming'],
 'CLICOLOR': ['1', '1', '1', '1'],
 'COMMONPROGRAMFILES': ['C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files'],
 'COMMONPROGRAMFILES(X86)': ['C:\\Program Files (x86)\\Common Files',
  'C:\\Program Files (x86)\\Common Files',
  'C:\\Program Files (x86)\\Common Files',
  'C:\\Program Files (x86)\\Common Files'],
 'COMMONPROGRAMW6432': ['C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files',
  'C:\\Program Files\\Common Files'],
 'COMPUTERNAME': ['LAPTOP-23GU92LK',
  'LAPTOP-23GU92LK',
  'LAPTOP-23GU92LK',
  'LAPTOP-23GU92LK'],
 'COMSPEC': ['C:\\WIND

## Creating the job and using Multiprocessing

In [7]:
from multiprocessing import Process, Manager

def f(d):
    d[1] = img
    d[2] = command_params
    d[3] = env_vars
    d[4] = cpu_memory_vars

In [8]:
if __name__ == '__main__':
    client = Manager()

    d = client.dict()
    d[1] = img
    d[2] = command_params
    d[3] = env_vars
    d[4] = cpu_memory_vars

    p1 = Process(target=f, args=(d,))
    p2 = Process(target=f, args=(d,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()

    print('Combining the parameters:', d)

Combining the parameters: {1: ['img1'], 2: ['cp', 'mv', 'rm'], 3: {'ALLUSERSPROFILE': ['C:\\ProgramData', 'C:\\ProgramData', 'C:\\ProgramData', 'C:\\ProgramData'], 'APPDATA': ['C:\\Users\\suneelnair1\\AppData\\Roaming', 'C:\\Users\\suneelnair1\\AppData\\Roaming', 'C:\\Users\\suneelnair1\\AppData\\Roaming', 'C:\\Users\\suneelnair1\\AppData\\Roaming'], 'COMMONPROGRAMFILES': ['C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files'], 'COMMONPROGRAMFILES(X86)': ['C:\\Program Files (x86)\\Common Files', 'C:\\Program Files (x86)\\Common Files', 'C:\\Program Files (x86)\\Common Files', 'C:\\Program Files (x86)\\Common Files'], 'COMMONPROGRAMW6432': ['C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files', 'C:\\Program Files\\Common Files'], 'COMPUTERNAME': ['LAPTOP-23GU92LK', 'LAPTOP-23GU92LK', 'LAPTOP-23GU92LK', 'LAPTOP-23GU92LK'], 'COMSPEC': ['C:\\WINDOWS\\system

## Parallel processing of jobs

In [9]:
def func1():
    print('func1: starting')
    for i in range(len(env_vars)): pass
    print(env_vars)

In [10]:
def func2():
    print('func2: starting')
    for i in range(cpu_memory_vars): pass
    print(cpu_memory_vars)

In [11]:
def runInParallel(*fns):
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        proc.append(p)
    for p in proc:
        p.join()        

In [12]:
runInParallel(func1, func2)

In [13]:
def func3(img,command_params,env_vars,cpu_memory_vars):
    transaction_count = len(env_vars)*len(command_params)
    return transaction_count

In [14]:
result = func3(img,command_params,env_vars,cpu_memory_vars)
print(result)

138


## Setup the Worker

In [15]:
import os
from redis import Redis
from rq import Worker, Queue, Connection

listen = ['default']

#redis_url = os.getenv('REDISTOGO_URL','redis://localhost:6379')
redis_conn = Redis()


if __name__ == '__main__':
    with Connection(redis_conn):
        worker = Worker(list(map(Queue,listen)))
        worker.work()


ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

In [16]:
from rq import Queue
from rq.job import Job
from redis import Redis
from rq import Worker


conn = Redis()

q = Queue(connection=conn)

def func3(img,command_params,env_vars,cpu_memory_vars):
    transaction_count = len(env_vars)*len(command_params)
    return transaction_count



## Create the job

In [17]:
job = q.enqueue_call(
            func=func3, args=(img,command_params,env_vars,cpu_memory_vars), result_ttl=5000
        )

print(job.get_id())

ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

## Return and save the results

In [20]:
# @app.route("/results/<job_key>", methods=['GET'])
def get_results(job_key):
 
     job = Job.fetch(job_key, connection=conn)
 
     if job.is_finished:
         return str(job.result), 200
     else:
        return "Job not finished", 202